# Stream Learning Classification Dataset 1000

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from river.metrics import Accuracy, BalancedAccuracy, CohenKappa, GeometricMean, Precision, Recall, F1
from river.tree import HoeffdingAdaptiveTreeClassifier, HoeffdingTreeClassifier
from river.forest import ARFClassifier
from river.ensemble import LeveragingBaggingClassifier
from river.ensemble import SRPClassifier
import math
import numpy as np

# Prequential Evaluation and Test

In [4]:
# Load the dataset into a Pandas DataFrame
df = pd.read_csv('data/dataset1000.csv')

# Convert the target variable to numerical values ('yes' replaced with 1 and 'no' replaced with 0)
le = LabelEncoder()
df['label_pass'] = le.fit_transform(df['label_pass'])

## Hoeffding Tree

In [5]:
# Define the number of iterations for the loop
n_iterations = 10

# Define lists to store the accuracy metrics for each iteration
accuracy_list_val = []
balanced_accuracy_list_val = []
geometric_mean_list_val = []
cohen_kappa_list_val = []
accuracy_list_test = []
balanced_accuracy_list_test = []
geometric_mean_list_test = []
cohen_kappa_list_test = []
precision_list_test = []
recall_list_test = []
f1_list_test = []

batch_size = 100
num_batches = math.ceil(len(df) / batch_size)
batches = [df[i*batch_size:(i+1)*batch_size] for i in range(num_batches-1)]  # List of dataframes
batches.append(df[(num_batches-1)*batch_size:])  # Appends a final dataframe, which contains any remaining rows that were not included in the previous batches

# Loop over the seed values
for seed in range(n_iterations):
    model = HoeffdingAdaptiveTreeClassifier(split_criterion='gini', seed=seed)

    accuracy_val = Accuracy()
    balanced_accuracy_val = BalancedAccuracy()
    geometric_mean_val = GeometricMean()
    cohen_kappa_val = CohenKappa()

    # Prequential Evaluation
    for batch in batches[:-1]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_val.update(y_true=y, y_pred=y_p)
                balanced_accuracy_val.update(y_true=y, y_pred=y_p)
                geometric_mean_val.update(y_true=y, y_pred=y_p)
                cohen_kappa_val.update(y_true=y, y_pred=y_p)
            model.learn_one(x, y)  # Train the model

        # Append the accuracy score to the list
        accuracy_list_val.append(accuracy_val.get())
        balanced_accuracy_list_val.append(balanced_accuracy_val.get())
        geometric_mean_list_val.append(geometric_mean_val.get())
        cohen_kappa_list_val.append(cohen_kappa_val.get())

    accuracy_test = Accuracy()
    balanced_accuracy_test = BalancedAccuracy()
    geometric_mean_test = GeometricMean()
    cohen_kappa_test = CohenKappa()
    precision_test = Precision()
    recall_test = Recall()
    f1_test = F1()

    for batch in batches[-1:]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_test.update(y_true=y, y_pred=y_p)
                balanced_accuracy_test.update(y_true=y, y_pred=y_p)
                geometric_mean_test.update(y_true=y, y_pred=y_p)
                cohen_kappa_test.update(y_true=y, y_pred=y_p)
                precision_test.update(y_true=y, y_pred=y_p)
                recall_test.update(y_true=y, y_pred=y_p)
                f1_test.update(y_true=y, y_pred=y_p)

        # Append the accuracy score to the list
        accuracy_list_test.append(accuracy_test.get())
        balanced_accuracy_list_test.append(balanced_accuracy_test.get())
        geometric_mean_list_test.append(geometric_mean_test.get())
        cohen_kappa_list_test.append(cohen_kappa_test.get())
        precision_list_test.append(precision_test.get())
        recall_list_test.append(recall_test.get())
        f1_list_test.append(f1_test.get())

# Calculate the mean and standard deviation of the accuracy metrics
accuracy_mean_val = np.mean(accuracy_list_val)
accuracy_std_val = np.std(accuracy_list_val)
balanced_accuracy_mean_val = np.mean(balanced_accuracy_list_val)
balanced_accuracy_std_val = np.std(balanced_accuracy_list_val)
geometric_mean_mean_val = np.mean(geometric_mean_list_val)
geometric_mean_std_val = np.std(geometric_mean_list_val)
cohen_kappa_mean_val = np.mean(cohen_kappa_list_val)
cohen_kappa_std_val = np.std(cohen_kappa_list_val)
accuracy_mean_test = np.mean(accuracy_list_test)
accuracy_std_test = np.std(accuracy_list_test)
balanced_accuracy_mean_test = np.mean(balanced_accuracy_list_test)
balanced_accuracy_std_test = np.std(balanced_accuracy_list_test)
geometric_mean_mean_test = np.mean(geometric_mean_list_test)
geometric_mean_std_test = np.std(geometric_mean_list_test)
cohen_kappa_mean_test = np.mean(cohen_kappa_list_test)
cohen_kappa_std_test = np.std(cohen_kappa_list_test)
precision_mean_test = np.mean(precision_list_test)
precision_std_test = np.std(precision_list_test)
recall_mean_test = np.mean(recall_list_test)
recall_std_test = np.std(recall_list_test)
f1_mean_test = np.mean(f1_list_test)
f1_std_test = np.std(f1_list_test)

# Print the mean and standard deviation of the accuracy metrics
print(f"Model accuracies w.r.t Prequential Evaluation 90% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_val:.2f} ± {accuracy_std_val:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_val:.2f} ± {balanced_accuracy_std_val:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_val:.2f} ± {geometric_mean_std_val:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_val:.2f} ± {cohen_kappa_std_val:.2f}")
print(f"Model accuracies w.r.t Test Set 10% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_test:.2f} ± {accuracy_std_test:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_test:.2f} ± {balanced_accuracy_std_test:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_test:.2f} ± {geometric_mean_std_test:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_test:.2f} ± {cohen_kappa_std_test:.2f}")
print(f"Precision: {precision_mean_test:.2f} ± {precision_std_test:.2f}")
print(f"Recall: {recall_mean_test:.2f} ± {recall_std_test:.2f}")
print(f"F1 Score: {f1_mean_test:.2f} ± {f1_std_test:.2f}")

Model accuracies w.r.t Prequential Evaluation 90% averaged over 10 iterations
Accuracy: 0.75 ± 0.02
Balanced Accuracy: 0.75 ± 0.02
Geometric Mean: 0.75 ± 0.02
Cohen's Kappa: 0.50 ± 0.04
Model accuracies w.r.t Test Set 10% averaged over 10 iterations
Accuracy: 0.67 ± 0.01
Balanced Accuracy: 0.67 ± 0.01
Geometric Mean: 0.67 ± 0.01
Cohen's Kappa: 0.34 ± 0.02
Precision: 0.71 ± 0.01
Recall: 0.59 ± 0.01
F1 Score: 0.65 ± 0.01


## Leveraging Bagging

In [6]:
# Define the number of iterations for the loop
n_iterations = 10

# Define lists to store the accuracy metrics for each iteration
accuracy_list_val = []
balanced_accuracy_list_val = []
geometric_mean_list_val = []
cohen_kappa_list_val = []
accuracy_list_test = []
balanced_accuracy_list_test = []
geometric_mean_list_test = []
cohen_kappa_list_test = []
precision_list_test = []
recall_list_test = []
f1_list_test = []

batch_size = 100
num_batches = math.ceil(len(df) / batch_size)
batches = [df[i*batch_size:(i+1)*batch_size] for i in range(num_batches-1)]  # List of dataframes
batches.append(df[(num_batches-1)*batch_size:])  # Appends a final dataframe, which contains any remaining rows that were not included in the previous batches

# Loop over the seed values
for seed in range(n_iterations):
    model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(split_criterion='gini'), n_models=10, seed=seed)

    accuracy_val = Accuracy()
    balanced_accuracy_val = BalancedAccuracy()
    geometric_mean_val = GeometricMean()
    cohen_kappa_val = CohenKappa()

    # Prequential Evaluation
    for batch in batches[:-1]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_val.update(y_true=y, y_pred=y_p)
                balanced_accuracy_val.update(y_true=y, y_pred=y_p)
                geometric_mean_val.update(y_true=y, y_pred=y_p)
                cohen_kappa_val.update(y_true=y, y_pred=y_p)
            model.learn_one(x, y)  # Train the model

        # Append the accuracy score to the list
        accuracy_list_val.append(accuracy_val.get())
        balanced_accuracy_list_val.append(balanced_accuracy_val.get())
        geometric_mean_list_val.append(geometric_mean_val.get())
        cohen_kappa_list_val.append(cohen_kappa_val.get())

    accuracy_test = Accuracy()
    balanced_accuracy_test = BalancedAccuracy()
    geometric_mean_test = GeometricMean()
    cohen_kappa_test = CohenKappa()
    precision_test = Precision()
    recall_test = Recall()
    f1_test = F1()

    for batch in batches[-1:]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_test.update(y_true=y, y_pred=y_p)
                balanced_accuracy_test.update(y_true=y, y_pred=y_p)
                geometric_mean_test.update(y_true=y, y_pred=y_p)
                cohen_kappa_test.update(y_true=y, y_pred=y_p)
                precision_test.update(y_true=y, y_pred=y_p)
                recall_test.update(y_true=y, y_pred=y_p)
                f1_test.update(y_true=y, y_pred=y_p)

        # Append the accuracy score to the list
        accuracy_list_test.append(accuracy_test.get())
        balanced_accuracy_list_test.append(balanced_accuracy_test.get())
        geometric_mean_list_test.append(geometric_mean_test.get())
        cohen_kappa_list_test.append(cohen_kappa_test.get())
        precision_list_test.append(precision_test.get())
        recall_list_test.append(recall_test.get())
        f1_list_test.append(f1_test.get())

# Calculate the mean and standard deviation of the accuracy metrics
accuracy_mean_val = np.mean(accuracy_list_val)
accuracy_std_val = np.std(accuracy_list_val)
balanced_accuracy_mean_val = np.mean(balanced_accuracy_list_val)
balanced_accuracy_std_val = np.std(balanced_accuracy_list_val)
geometric_mean_mean_val = np.mean(geometric_mean_list_val)
geometric_mean_std_val = np.std(geometric_mean_list_val)
cohen_kappa_mean_val = np.mean(cohen_kappa_list_val)
cohen_kappa_std_val = np.std(cohen_kappa_list_val)
accuracy_mean_test = np.mean(accuracy_list_test)
accuracy_std_test = np.std(accuracy_list_test)
balanced_accuracy_mean_test = np.mean(balanced_accuracy_list_test)
balanced_accuracy_std_test = np.std(balanced_accuracy_list_test)
geometric_mean_mean_test = np.mean(geometric_mean_list_test)
geometric_mean_std_test = np.std(geometric_mean_list_test)
cohen_kappa_mean_test = np.mean(cohen_kappa_list_test)
cohen_kappa_std_test = np.std(cohen_kappa_list_test)
precision_mean_test = np.mean(precision_list_test)
precision_std_test = np.std(precision_list_test)
recall_mean_test = np.mean(recall_list_test)
recall_std_test = np.std(recall_list_test)
f1_mean_test = np.mean(f1_list_test)
f1_std_test = np.std(f1_list_test)

# Print the mean and standard deviation of the accuracy metrics
print(f"Model accuracies w.r.t Prequential Evaluation 90% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_val:.2f} ± {accuracy_std_val:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_val:.2f} ± {balanced_accuracy_std_val:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_val:.2f} ± {geometric_mean_std_val:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_val:.2f} ± {cohen_kappa_std_val:.2f}")
print(f"Model accuracies w.r.t Test Set 10% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_test:.2f} ± {accuracy_std_test:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_test:.2f} ± {balanced_accuracy_std_test:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_test:.2f} ± {geometric_mean_std_test:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_test:.2f} ± {cohen_kappa_std_test:.2f}")
print(f"Precision: {precision_mean_test:.2f} ± {precision_std_test:.2f}")
print(f"Recall: {recall_mean_test:.2f} ± {recall_std_test:.2f}")
print(f"F1 Score: {f1_mean_test:.2f} ± {f1_std_test:.2f}")

Model accuracies w.r.t Prequential Evaluation 90% averaged over 10 iterations
Accuracy: 0.76 ± 0.02
Balanced Accuracy: 0.76 ± 0.03
Geometric Mean: 0.76 ± 0.02
Cohen's Kappa: 0.52 ± 0.05
Model accuracies w.r.t Test Set 10% averaged over 10 iterations
Accuracy: 0.75 ± 0.01
Balanced Accuracy: 0.75 ± 0.01
Geometric Mean: 0.75 ± 0.01
Cohen's Kappa: 0.50 ± 0.02
Precision: 0.77 ± 0.03
Recall: 0.74 ± 0.03
F1 Score: 0.75 ± 0.01


## Adaptive Random Forest

In [7]:
# Define the number of iterations for the loop
n_iterations = 10

# Define lists to store the accuracy metrics for each iteration
accuracy_list_val = []
balanced_accuracy_list_val = []
geometric_mean_list_val = []
cohen_kappa_list_val = []
accuracy_list_test = []
balanced_accuracy_list_test = []
geometric_mean_list_test = []
cohen_kappa_list_test = []
precision_list_test = []
recall_list_test = []
f1_list_test = []

batch_size = 100
num_batches = math.ceil(len(df) / batch_size)
batches = [df[i*batch_size:(i+1)*batch_size] for i in range(num_batches-1)]  # List of dataframes
batches.append(df[(num_batches-1)*batch_size:])  # Appends a final dataframe, which contains any remaining rows that were not included in the previous batches

# Loop over the seed values
for seed in range(n_iterations):
    model = ARFClassifier(n_models=10, split_criterion='gini', seed=seed)
    accuracy_val = Accuracy()
    balanced_accuracy_val = BalancedAccuracy()
    geometric_mean_val = GeometricMean()
    cohen_kappa_val = CohenKappa()

    # Prequential Evaluation
    for batch in batches[:-1]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_val.update(y_true=y, y_pred=y_p)
                balanced_accuracy_val.update(y_true=y, y_pred=y_p)
                geometric_mean_val.update(y_true=y, y_pred=y_p)
                cohen_kappa_val.update(y_true=y, y_pred=y_p)
            model.learn_one(x, y)  # Train the model

        # Append the accuracy score to the list
        accuracy_list_val.append(accuracy_val.get())
        balanced_accuracy_list_val.append(balanced_accuracy_val.get())
        geometric_mean_list_val.append(geometric_mean_val.get())
        cohen_kappa_list_val.append(cohen_kappa_val.get())

    accuracy_test = Accuracy()
    balanced_accuracy_test = BalancedAccuracy()
    geometric_mean_test = GeometricMean()
    cohen_kappa_test = CohenKappa()
    precision_test = Precision()
    recall_test = Recall()
    f1_test = F1()

    for batch in batches[-1:]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_test.update(y_true=y, y_pred=y_p)
                balanced_accuracy_test.update(y_true=y, y_pred=y_p)
                geometric_mean_test.update(y_true=y, y_pred=y_p)
                cohen_kappa_test.update(y_true=y, y_pred=y_p)
                precision_test.update(y_true=y, y_pred=y_p)
                recall_test.update(y_true=y, y_pred=y_p)
                f1_test.update(y_true=y, y_pred=y_p)

        # Append the accuracy score to the list
        accuracy_list_test.append(accuracy_test.get())
        balanced_accuracy_list_test.append(balanced_accuracy_test.get())
        geometric_mean_list_test.append(geometric_mean_test.get())
        cohen_kappa_list_test.append(cohen_kappa_test.get())
        precision_list_test.append(precision_test.get())
        recall_list_test.append(recall_test.get())
        f1_list_test.append(f1_test.get())

# Calculate the mean and standard deviation of the accuracy metrics
accuracy_mean_val = np.mean(accuracy_list_val)
accuracy_std_val = np.std(accuracy_list_val)
balanced_accuracy_mean_val = np.mean(balanced_accuracy_list_val)
balanced_accuracy_std_val = np.std(balanced_accuracy_list_val)
geometric_mean_mean_val = np.mean(geometric_mean_list_val)
geometric_mean_std_val = np.std(geometric_mean_list_val)
cohen_kappa_mean_val = np.mean(cohen_kappa_list_val)
cohen_kappa_std_val = np.std(cohen_kappa_list_val)
accuracy_mean_test = np.mean(accuracy_list_test)
accuracy_std_test = np.std(accuracy_list_test)
balanced_accuracy_mean_test = np.mean(balanced_accuracy_list_test)
balanced_accuracy_std_test = np.std(balanced_accuracy_list_test)
geometric_mean_mean_test = np.mean(geometric_mean_list_test)
geometric_mean_std_test = np.std(geometric_mean_list_test)
cohen_kappa_mean_test = np.mean(cohen_kappa_list_test)
cohen_kappa_std_test = np.std(cohen_kappa_list_test)
precision_mean_test = np.mean(precision_list_test)
precision_std_test = np.std(precision_list_test)
recall_mean_test = np.mean(recall_list_test)
recall_std_test = np.std(recall_list_test)
f1_mean_test = np.mean(f1_list_test)
f1_std_test = np.std(f1_list_test)

# Print the mean and standard deviation of the accuracy metrics
print(f"Model accuracies w.r.t Prequential Evaluation 90% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_val:.2f} ± {accuracy_std_val:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_val:.2f} ± {balanced_accuracy_std_val:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_val:.2f} ± {geometric_mean_std_val:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_val:.2f} ± {cohen_kappa_std_val:.2f}")
print(f"Model accuracies w.r.t Test Set 10% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_test:.2f} ± {accuracy_std_test:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_test:.2f} ± {balanced_accuracy_std_test:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_test:.2f} ± {geometric_mean_std_test:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_test:.2f} ± {cohen_kappa_std_test:.2f}")
print(f"Precision: {precision_mean_test:.2f} ± {precision_std_test:.2f}")
print(f"Recall: {recall_mean_test:.2f} ± {recall_std_test:.2f}")
print(f"F1 Score: {f1_mean_test:.2f} ± {f1_std_test:.2f}")

Model accuracies w.r.t Prequential Evaluation 90% averaged over 10 iterations
Accuracy: 0.76 ± 0.02
Balanced Accuracy: 0.76 ± 0.02
Geometric Mean: 0.75 ± 0.02
Cohen's Kappa: 0.51 ± 0.04
Model accuracies w.r.t Test Set 10% averaged over 10 iterations
Accuracy: 0.79 ± 0.03
Balanced Accuracy: 0.79 ± 0.03
Geometric Mean: 0.78 ± 0.03
Cohen's Kappa: 0.57 ± 0.06
Precision: 0.77 ± 0.03
Recall: 0.84 ± 0.03
F1 Score: 0.80 ± 0.03


## StreamingRandomPatches

In [8]:
# Define the number of iterations for the loop
n_iterations = 10

# Define lists to store the accuracy metrics for each iteration
accuracy_list_val = []
balanced_accuracy_list_val = []
geometric_mean_list_val = []
cohen_kappa_list_val = []
accuracy_list_test = []
balanced_accuracy_list_test = []
geometric_mean_list_test = []
cohen_kappa_list_test = []
precision_list_test = []
recall_list_test = []
f1_list_test = []

batch_size = 100
num_batches = math.ceil(len(df) / batch_size)
batches = [df[i*batch_size:(i+1)*batch_size] for i in range(num_batches-1)]  # List of dataframes
batches.append(df[(num_batches-1)*batch_size:])  # Appends a final dataframe, which contains any remaining rows that were not included in the previous batches

# Loop over the seed values
for seed in range(n_iterations):
    model = SRPClassifier(model=HoeffdingTreeClassifier(split_criterion='gini'), n_models=10, seed=seed)
    accuracy_val = Accuracy()
    balanced_accuracy_val = BalancedAccuracy()
    geometric_mean_val = GeometricMean()
    cohen_kappa_val = CohenKappa()

    # Prequential Evaluation
    for batch in batches[:-1]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_val.update(y_true=y, y_pred=y_p)
                balanced_accuracy_val.update(y_true=y, y_pred=y_p)
                geometric_mean_val.update(y_true=y, y_pred=y_p)
                cohen_kappa_val.update(y_true=y, y_pred=y_p)
            model.learn_one(x, y)  # Train the model

        # Append the accuracy score to the list
        accuracy_list_val.append(accuracy_val.get())
        balanced_accuracy_list_val.append(balanced_accuracy_val.get())
        geometric_mean_list_val.append(geometric_mean_val.get())
        cohen_kappa_list_val.append(cohen_kappa_val.get())

    accuracy_test = Accuracy()
    balanced_accuracy_test = BalancedAccuracy()
    geometric_mean_test = GeometricMean()
    cohen_kappa_test = CohenKappa()
    precision_test = Precision()
    recall_test = Recall()
    f1_test = F1()

    for batch in batches[-1:]:
        for index, row in batch.iterrows():
            x = row[['|v|_mean', '|a|_mean', 'vz_mean', 'az_mean', '|v|_std', '|a|_std', 'vz_std', 'az_std', '|v|_min',
                     '|a|_min', 'vz_min', 'az_min', '|v|_max', '|a|_max', 'vz_max', 'az_max']]
            y = row['label_pass']

            y_p = model.predict_one(x)  # Predict class
            if y_p is not None:
                accuracy_test.update(y_true=y, y_pred=y_p)
                balanced_accuracy_test.update(y_true=y, y_pred=y_p)
                geometric_mean_test.update(y_true=y, y_pred=y_p)
                cohen_kappa_test.update(y_true=y, y_pred=y_p)
                precision_test.update(y_true=y, y_pred=y_p)
                recall_test.update(y_true=y, y_pred=y_p)
                f1_test.update(y_true=y, y_pred=y_p)

        # Append the accuracy score to the list
        accuracy_list_test.append(accuracy_test.get())
        balanced_accuracy_list_test.append(balanced_accuracy_test.get())
        geometric_mean_list_test.append(geometric_mean_test.get())
        cohen_kappa_list_test.append(cohen_kappa_test.get())
        precision_list_test.append(precision_test.get())
        recall_list_test.append(recall_test.get())
        f1_list_test.append(f1_test.get())

# Calculate the mean and standard deviation of the accuracy metrics
accuracy_mean_val = np.mean(accuracy_list_val)
accuracy_std_val = np.std(accuracy_list_val)
balanced_accuracy_mean_val = np.mean(balanced_accuracy_list_val)
balanced_accuracy_std_val = np.std(balanced_accuracy_list_val)
geometric_mean_mean_val = np.mean(geometric_mean_list_val)
geometric_mean_std_val = np.std(geometric_mean_list_val)
cohen_kappa_mean_val = np.mean(cohen_kappa_list_val)
cohen_kappa_std_val = np.std(cohen_kappa_list_val)
accuracy_mean_test = np.mean(accuracy_list_test)
accuracy_std_test = np.std(accuracy_list_test)
balanced_accuracy_mean_test = np.mean(balanced_accuracy_list_test)
balanced_accuracy_std_test = np.std(balanced_accuracy_list_test)
geometric_mean_mean_test = np.mean(geometric_mean_list_test)
geometric_mean_std_test = np.std(geometric_mean_list_test)
cohen_kappa_mean_test = np.mean(cohen_kappa_list_test)
cohen_kappa_std_test = np.std(cohen_kappa_list_test)
precision_mean_test = np.mean(precision_list_test)
precision_std_test = np.std(precision_list_test)
recall_mean_test = np.mean(recall_list_test)
recall_std_test = np.std(recall_list_test)
f1_mean_test = np.mean(f1_list_test)
f1_std_test = np.std(f1_list_test)

# Print the mean and standard deviation of the accuracy metrics
print(f"Model accuracies w.r.t Prequential Evaluation 90% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_val:.2f} ± {accuracy_std_val:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_val:.2f} ± {balanced_accuracy_std_val:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_val:.2f} ± {geometric_mean_std_val:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_val:.2f} ± {cohen_kappa_std_val:.2f}")
print(f"Model accuracies w.r.t Test Set 10% averaged over {n_iterations} iterations")
print(f"Accuracy: {accuracy_mean_test:.2f} ± {accuracy_std_test:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_mean_test:.2f} ± {balanced_accuracy_std_test:.2f}")
print(f"Geometric Mean: {geometric_mean_mean_test:.2f} ± {geometric_mean_std_test:.2f}")
print(f"Cohen's Kappa: {cohen_kappa_mean_test:.2f} ± {cohen_kappa_std_test:.2f}")
print(f"Precision: {precision_mean_test:.2f} ± {precision_std_test:.2f}")
print(f"Recall: {recall_mean_test:.2f} ± {recall_std_test:.2f}")
print(f"F1 Score: {f1_mean_test:.2f} ± {f1_std_test:.2f}")

Model accuracies w.r.t Prequential Evaluation 90% averaged over 10 iterations
Accuracy: 0.76 ± 0.02
Balanced Accuracy: 0.76 ± 0.02
Geometric Mean: 0.76 ± 0.02
Cohen's Kappa: 0.52 ± 0.04
Model accuracies w.r.t Test Set 10% averaged over 10 iterations
Accuracy: 0.77 ± 0.02
Balanced Accuracy: 0.77 ± 0.02
Geometric Mean: 0.77 ± 0.02
Cohen's Kappa: 0.54 ± 0.04
Precision: 0.82 ± 0.02
Recall: 0.70 ± 0.03
F1 Score: 0.75 ± 0.02
